install required packages

In [1]:
using Pkg;
#Pkg.add("GDAL");

loading packages

In [11]:
using ESDL
using DelimitedFiles
using Statistics

load the data cube

In [3]:
c = Cube("/home/jovyan/work/datacube/ESDCv2.0.0/esdc-8d-0.083deg-184x270x270-2.0.0.zarr/")

Collection of ZArray Cube with the following dimensions
Lon                 Axis with 4320 Elements from -179.95833 to 179.9569
Lat                 Axis with 2160 Elements from 89.958336 to -89.95761
Time                Axis with 1702 Elements from 1980-01-05 to 2016-12-30
Variable            Axis with 75 elements: soil_moisture leaf_area_index .. snow_sublimation Rg 
Total size: 5.42 TB


subset the cube concerning time and location but keep all variables

In [8]:
d = subsetcube(c, Lon = (73, 105), 
                  Lat = (25, 40), 
                  time = (Date(2000,1,1), Date(2018,12,31))
    )

Collection of ZArray Cube with the following dimensions
Lon                 Axis with 384 Elements from 73.04066 to 104.95719
Lat                 Axis with 180 Elements from 39.958534 to 25.04193
Time                Axis with 782 Elements from 2000-01-05 to 2016-12-30
Variable            Axis with 75 elements: soil_moisture leaf_area_index .. snow_sublimation Rg 
Total size: 18.88 GB


calculate mean value for all variables and the full time period considered

In [30]:
d2 = mapslices(maximum, d, dims=("Time"))

┌ Warning: There are still cache misses
└ @ ESDL.DAT /home/jovyan/julia_depots/packages/ESDL/kpnFC/src/DAT/DAT.jl:531
┌ Warning: There are compressed caches misses, you may want to use a different cube chunking
└ @ ESDL.DAT /home/jovyan/julia_depots/packages/ESDL/kpnFC/src/DAT/DAT.jl:532


TypeError: TypeError: non-boolean (Missing) used in boolean context

check access for individual variables and check which variables have missing data

In [26]:
d2[:,:,11:20]

384×180×10 Array{Union{Missing, Float32},3}:
[:, :, 1] =
 missing  missing  missing  missing  …  missing  missing  missing  missing
 missing  missing  missing  missing     missing  missing  missing  missing
 missing  missing  missing  missing     missing  missing  missing  missing
 missing  missing  missing  missing     missing  missing  missing  missing
 missing  missing  missing  missing     missing  missing  missing  missing
 missing  missing  missing  missing  …  missing  missing  missing  missing
 missing  missing  missing  missing     missing  missing  missing  missing
 missing  missing  missing  missing     missing  missing  missing  missing
 missing  missing  missing  missing     missing  missing  missing  missing
 missing  missing  missing  missing     missing  missing  missing  missing
 missing  missing  missing  missing  …  missing  missing  missing  missing
 missing  missing  missing  missing     missing  missing  missing  missing
 missing  missing  missing  missing     mis

store the variable names into an array

In [ ]:
vars = getAxis("Variable", d)

have a look at the variable names

In [ ]:
vars.values

extract longitude and latitude values of the cube grid

In [ ]:
lon = getAxis("Lon", d)

In [ ]:
lat = getAxis("Lat", d)

have a look at the lat and lon values

In [ ]:
lat.values

In [ ]:
lon.values

create dummy array with number of entries matching the total number of grid cells

In [ ]:
col1 = collect(1:length(lon.values)*length(lat.values))
col1 = col1*1.0

In [ ]:
create empty matrix with several colums matching the length of total number of grid cells

In [ ]:
mat = [col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1 col1]

create a sequence marking the starting points of each latitude column

In [ ]:
len = length(lat)
seq=[1:len:length(col1);]
append!(seq, length(col1))

write lat and lon value-combinations into the first two columns of the empty matrix

In [ ]:
i = 1
for j in lon.values
  for j1 in lat.values
            mat[i,1:2] = [j j1]
            i = i + 1
        end
    end

In [ ]:
typeof(mat)

now write the actual values of the data variables into the remaining columns - so far still a problem with missing values; either fix this above by subsetting or interpolating the cube; alternatively replace missing values within the loop (no idea yet how to do this)

the last part has to be out of the loop due to the way it is currently coded...

In [ ]:
for j2 in 1:(length(lon.values)-1)
   
    mat[(seq[j2]:(seq[j2+1]-1)),3:5] = d[j2,:,1,4:6]
    print(j2)
        
end

mat[(seq[(length(seq)-1)]:(seq[length(seq)])),3:5] = d[length(lon.values),:,1,4:6]

In [ ]:
mat[:,1:6]